In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 100)

from sklearn.svm import SVC
from sklearn.utils import shuffle


# TODO: Change to larger training set

train_set = pd.read_csv('../Small Training Set.csv', header=None)

# train_set = pd.read_csv('../20 Percent Training Set.csv', header=None)
# train_set = pd.read_csv('../KDDTrain+.csv', header=None)

fn = pd.read_csv('../Field Names.csv', header=None)

fn2 = pd.DataFrame([['attack_type', 'symbolic'],['??','continuous']])
fn = fn.append(fn2, ignore_index=True)

cont_feats = fn.loc[fn[1] == 'continuous']
cont_feats = cont_feats.drop(columns=[1])

fn = fn.drop(columns=[1])

dfhd = fn[0].tolist()
train_set.columns = dfhd

cols = train_set.columns.tolist()
cols = cols[-2:-1] + cols[:-2] + cols[-1:]
train_set = train_set[cols]

att_types = train_set['attack_type']

# train_set.sort_values('attack_type')

In [2]:
# ENUMERATING SERVICE TYPE
import copy

service_list = train_set['service'].unique()

try:
    service_list2
except NameError:
    service_list2 = copy.deepcopy(service_list)    
    svc_enum_data = {'service': service_list}
    service_enumerations = pd.DataFrame(data=svc_enum_data)
    service_enumerations.to_csv('Service_enums.csv')

enumerated_service_list = [x for x in range(len(service_list))]


train_set = train_set.replace(service_list,enumerated_service_list)


In [3]:
# ENUMERATING PROTOCOL TYPE

prot_list = train_set['protocol_type'].unique()

try:
    prot_list2
except NameError:
    prot_list2 = copy.deepcopy(prot_list)    
    prot_enum_data = {'protocol_type': prot_list}
    prot_enumerations = pd.DataFrame(data=prot_enum_data)
    prot_enumerations.to_csv('Prot_enums.csv')

enumerated_prot_list = [x for x in range(len(prot_list))]


train_set = train_set.replace(prot_list,enumerated_prot_list)


In [4]:
# ENUMERATING FLAG TYPE

flag_list = train_set['flag'].unique()

try:
    flag_list2
except NameError:
    flag_list2 = copy.deepcopy(flag_list)    
    flag_enum_data = {'flag': flag_list}
    flag_enumerations = pd.DataFrame(data=flag_enum_data)
    flag_enumerations.to_csv('Flag_enums.csv')

enumerated_flag_list = [x for x in range(len(flag_list))]


train_set = train_set.replace(flag_list,enumerated_flag_list)

In [5]:
clf = SVC(kernel='linear')
# clf = SVC()

train_features = train_set.drop(columns=['attack_type'])
train_target = train_set['attack_type']

from sklearn import preprocessing

# Use a min-max scaler?

# min_max_scaler = preprocessing.MinMaxScaler()

# train_feats_scaled = min_max_scaler.fit_transform(train_features.values)
# train_features.iloc[:,:] = train_feats_scaled

# scaler = preprocessing.MaxAbsScaler()

# train_feats_scaled = scaler.fit_transform(train_features.values)
# train_features.iloc[:,:] = train_feats_scaled


clf.fit(train_features, train_target)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [10]:
test_set = pd.read_csv('../KDDTest+.csv', header=None)
test_set.columns = dfhd

test_set = test_set[cols]

test_set = test_set.replace(service_list2,enumerated_service_list)
test_set['service'] = test_set['service'].apply(lambda x: '-1' if type(x) is str else x)

test_set = test_set.replace(prot_list2, enumerated_prot_list)
test_set['protocol_type'] = test_set['protocol_type'].apply(lambda x: '-1' if type(x) is str else x)

test_set = test_set.replace(flag_list2, enumerated_flag_list)
test_set['flag'] = test_set['flag'].apply(lambda x: '-1' if type(x) is str else x)

# Mini test_set as not full training set used
test_set = shuffle(test_set)
test_set = test_set[:201]


test_features = test_set.drop(columns=['attack_type'])
test_target = test_set['attack_type']


# test_feats_scaled = min_max_scaler.fit_transform(test_features.values)
# test_features.iloc[:,:] = test_feats_scaled

# test_feats_scaled = scaler.fit_transform(test_features.values)
# test_features.iloc[:,:] = test_feats_scaled



pred_list = clf.predict(test_features)

np.mean( pred_list == test_target)

0.7064676616915423